# Cortex AI 関数: マルチモーダルデータ向けAIクエリ言語としてのSQLの再構築


In [ ]:
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.context import get_active_session

session = get_active_session()

## テキストおよび画像データ形式にわたる顧客の問題を特定する

* Text: サポート Emails 
* Images: バグレポート
* Audio: ボイスメール


同一のSQL演算子を使用し、あらゆるモダリティをシームレスに横断して動作

*使用関数: AI_COMPLETE(), AI_TRANSCRIBE()*

In [ ]:
CREATE OR REPLACE TABLE insights as
with IMAGE_INSIGHTS as (
    SELECT
        created_at,
        user_id,
        relative_path as ticket_id,
        img_file as input_file,
        file_url as input_file_url,
        AI_COMPLETE('gemini-3-pro', prompt('このスクリーンショットに表示されている問題を、簡潔な一文で要約してください: {0}', img_file))::VARCHAR as summary,
        summary as content
    from
        images
    ),
EMAIL_INSIGHTS as (
    SELECT
        created_at,
        user_id,
        ticket_id::text as ticket_id,
        null as input_file,
        '' as input_file_url,
        content as content,
        AI_COMPLETE('claude-4-sonnet', prompt('この問題を簡潔な一文で要約してください。
ユーザーが音楽の好みに関する言及をしている場合は、その情報も含めてください: {0}', content))::VARCHAR as summary
    FROM
        emails
),
AUDIO_INSIGHTS as (
    SELECT
        created_at,
        user_id,
        relative_path as ticket_id,
        audio_file as input_file,
        file_url as input_file_url,
        AI_COMPLETE('claude-4-sonnet', prompt('この問題を簡潔な一文で要約してください: {0}', (AI_TRANSCRIBE(audio_file)['text'])))::VARCHAR as summary,
        summary as content
    FROM
        voicemails
)
SELECT
    'Image' as source,
    created_at,
    user_id,
    ticket_id,
    input_file,
    input_file_url,
    content,
    summary 
FROM
    IMAGE_INSIGHTS

UNION

SELECT
    'Email' as source,
    created_at,
    user_id,
    ticket_id,
    input_file,
    input_file_url,
    content,
    summary 
FROM
    EMAIL_INSIGHTS

UNION

SELECT
    'Voicemail' as source,
    created_at,
    user_id,
    ticket_id,
    input_file,
    input_file_url,
    content,
    summary 
FROM
    AUDIO_INSIGHTS
;

## テキスト、画像、音声データ形式を横断した統合データ

ネイティブFILEデータ型により、すべてのフォーマットを1つのテーブルに統合可能

In [ ]:
SELECT 
    user_id,
    source,
    input_file,
    summary,
    content,
    input_file_url 
FROM
    insights
ORDER BY
    input_file_url DESC
;

## 意味論的結合の問題と解決策ライブラリ

AI 関係を基盤として、顧客課題を既存ソリューションと効率的に「結合」する

*使用機能: ... JOIN ... ON AI_FILTER()*

In [ ]:
SELECT
    c.content as "CUSTOMER ISSUE",
    s.solution,
    c.created_at,
FROM
    INSIGHTS c
LEFT JOIN
    SOLUTION_CENTER_ARTICLES s
    ON
        AI_FILTER(prompt('お客様の問題内容と、ソリューションセンターの記事が提供されます。その解決記事が顧客の懸念に対応できるかを確認してください。※ エラーの詳細が一致しているかを必ず確認してください。: {0}; \n\nソリューション: {1}', content, s.solution))
ORDER BY
    CREATED_AT ASC
;

## 月別主要課題の集計

複数の行にわたる集計インサイトを取得する

*使用関数: AI_AGG()*

In [ ]:
AGGREGATE_PROMPT = """
サポートチケットのレビューをすべて分析し、言及されている問題点を漏れなく網羅した一覧を作成してください。
回答は、問題点ごとの箇条書きにし、それぞれの出現頻度（％）を添えてください。.
"""

sql = f"""
select 
    monthname(created_at) as month, 
    count(*) as total_tickets,
    count(distinct user_id) as unique_users,
    AI_AGG(summary,'{AGGREGATE_PROMPT}') as top_issues_reported,
    from (select * from insights order by random() limit 200)
    group by month
    order by total_tickets desc,month desc
"""

df = session.sql(sql).to_pandas()
rows_to_display = 1
for row in df[:rows_to_display].itertuples():
    st.subheader(f"Aggregated Insights for {row.MONTH}")
    st.metric("Total Tickets",row.TOTAL_TICKETS)
    st.metric("Unique Users", row.UNIQUE_USERS)
    st.subheader("Top Issues")
    st.markdown(row.TOP_ISSUES_REPORTED)
    st.divider()

df_long = df.melt(id_vars='MONTH', value_vars=['TOTAL_TICKETS', 'UNIQUE_USERS'],
                  var_name='Metric', value_name='Total')

chart = alt.Chart(df_long).mark_bar().encode(
    y=alt.Y('MONTH:N', sort='-x'),
    x=alt.X('Total:Q'),
    color=alt.Color('Metric:N', scale=alt.Scale(scheme='tableau10')),
    tooltip=['MONTH', 'Metric', 'Total']
).properties(height=300)

st.altair_chart(chart, use_container_width=True)

## Classification: 分類

下流アプリケーションで使用可能なラベルの分類を構築します。例えば、機械学習モデルのトレーニングに利用できます。

*Function used: AI_CLASSIFY()*

In [ ]:
SET FILTER_PROMPT = '
顧客のコメントの中に、音楽ジャンルの好みについて言及があるかを確認したいです。
このコメントには、顧客が特定の音楽ジャンルを好んでいることが示されていますか？: {0}';

SET CLASSIFY_PROMPT = 'このコメントに記載されている音楽の好みを分類するのを手伝ってください: ';
SET MUSIC_GENRES = 'Electronic/Dance Music (EDM), Jazz, Indie/Folk, Rock, Classical, World Music, Blues, Pop';

CREATE OR REPLACE TABLE filtered AS
SELECT
    *
FROM
    (select * from insights order by random() limit 500)
WHERE
    AI_FILTER(prompt($FILTER_PROMPT, summary))
;

SELECT
    source, 
    summary,
    AI_CLASSIFY($CLASSIFY_PROMPT || summary,SPLIT($MUSIC_GENRES, ','))['labels'][0] as classified_label
FROM
    filtered
;

In [ ]:
df = AI_CLASSIFY.to_pandas()

# Group by genre and calculate counts
genre_counts = df['CLASSIFIED_LABEL'].value_counts().reset_index()
genre_counts.columns = ['Genre', 'Count']

# Pie chart using Altair
chart = alt.Chart(genre_counts).mark_arc().encode(
    theta=alt.Theta(field='Count', type='quantitative'),
    color=alt.Color(field='Genre', type='nominal'),
    tooltip=['Genre', 'Count']
).properties(
    width=500,
    height=400
)

st.subheader('Distribution of Genres')
st.altair_chart(chart, use_container_width=True)